In [1]:
from src.rag.components.retriever import HybridRetriever

/Users/esp.py/Projects/Personal/end-to-end-rag/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
embedding_model_id = "dunzhang/stella_en_400M_v5"

In [3]:
from pathlib import Path

In [4]:
model_path = Path.cwd().joinpath("models")

In [5]:
embedding_model_path = model_path.joinpath(embedding_model_id ).__str__()

In [6]:
cross_encoder_kwargs = {
    "model_name": embedding_model_path,
    "trust_remote_code": True,
    "local_files_only": True,
    "config_kwargs": {"use_memory_efficient_attention": False, "unpad_inputs": False},
    "device": "cpu"
}

In [7]:

transformer_kwargs  = { "model_name_or_path": embedding_model_path,
    "trust_remote_code": True,
    "device": "cpu",
    "config_kwargs": {"use_memory_efficient_attention": False,
                   "unpad_inputs": False},
    "cache_folder": model_path}

In [8]:
spacy_model_id = "en_core_web_sm"

In [9]:
retriever = HybridRetriever(
    cross_encoder_kwargs=cross_encoder_kwargs,
    spacy_model=spacy_model_id,
    sentence_transformer_kwargs=transformer_kwargs,
    language="english"
)

Some weights of the model checkpoint at /Users/esp.py/Projects/Personal/end-to-end-rag/models/dunzhang/stella_en_400M_v5 were not used when initializing NewModel: ['new.pooler.dense.bias', 'new.pooler.dense.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of NewForSequenceClassification were not initialized from the model checkpoint at /Users/esp.py/Projects/Personal/end-to-end-rag/models/dunzhang/stella_en_400M_v5 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predi

In [10]:
query = "Can I apply for a First Time Buyer mortgage?"
# "How do you work out your LTV ratio?"

In [11]:
semantic_search_results = retriever.semantic_search(query=query, limit=10)

/Users/esp.py/Projects/Personal/end-to-end-rag/.venv/lib/python3.10/site-packages/transformers/modeling_utils.py:1141: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [12]:
keywords = retriever.perform_keyword_extraction(query)

In [13]:
keywords

'"Time Buyer mortgage"'

In [14]:
keywords_search_results = retriever.keyword_search(keywords=keywords, limit=10)

We can add a reranker but we decided to keep it simple for now.

In [15]:
semantic_search_results

[Record(id='828406b387ed9619b567ce29477db9f45cf9dfbeb09224c19704fca6bb3d0125', content='First Time Buyers Types of mortgage How to apply for a mortgage How does a mortgage work?'),
 Record(id='1c7d54049c9ecb6a38a8b376447ab26546b684b1d8ec5f4fd7df79a49343fccd', content='Can you get a buy to let mortgage as a first time buyer?                                  expandable section'),
 Record(id='fa5bf87eaf3cbd96e6c15c4cd276010977bb6108bc40ec197c178362d8cd26e4', content='Find out more about our first time buyer mortgage deals and how we can help you take your first step on the property ladder. Learn how to apply for an Agreement in Principle and work out how much you could borrow towards your first home, subject to a full mortgage application.'),
 Record(id='af1e0f7e7028915887222dbfdce106afc15557b6a1a16d6a179ce6cb915f8295', content='Our range of first time buyer mortgage deals could help you get the keys to your new home.'),
 Record(id='10ad30596bae6736439fbf384aa5e5ff0d32a4c3ee35dada2723b0a1

For some reason the encoder is taking long to return.

In [16]:
keywords_search_results

[Record(id='10ad30596bae6736439fbf384aa5e5ff0d32a4c3ee35dada2723b0a16a143347', content='You might come across the term first time buyer mortgage, or FTB mortgage. This is how we talk about mortgages for people who are buying a home for the first time. There are also some government mortgage schemes that are specifically for first time buyers. For example, the mortgage guarantee scheme may allow you to apply for a mortgage with a 5% deposit. Visit our first time buyer mortgage hub'),
 Record(id='0af82a8beff801a6be9496550173210710deef3531430ae922990f0aaabfcc14', content="We're proud to support the Government’s mortgage guarantee scheme. You can apply for a first time buyer mortgage of up to 95% of the property's value."),
 Record(id='67e9f3b42229902cd9995ba7e7920433a78253aeb2cfd0360f44a516cf46e62b', content="Use our mortgage calculator to help work out how much your mortgage repayments could be once you’ve paid a deposit. How much can I borrow as a first time buyer? Mortgage lenders may 

In [17]:
# scores = retriever.cross_encoder.predict(
#    [(query, item.content) for item in keywords_search_results + semantic_search_results])

### Generator

In [18]:
from src.rag.components.generator import  LLamaCppGeneratorComponent

In [19]:
prompt = "you are a helpful mortgage advisor"

In [20]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

In [21]:
llama_cpp_generator = LLamaCppGeneratorComponent(
    api_url="http://127.0.0.1:8001",
    model_name=model_name,
    prompt=prompt
)

In [22]:
llama_cpp_generator._ping_api()

True

In [23]:
documents = [doc.content for doc in keywords_search_results + semantic_search_results]

In [24]:
documents

['You might come across the term first time buyer mortgage, or FTB mortgage. This is how we talk about mortgages for people who are buying a home for the first time. There are also some government mortgage schemes that are specifically for first time buyers. For example, the mortgage guarantee scheme may allow you to apply for a mortgage with a 5% deposit. Visit our first time buyer mortgage hub',
 "We're proud to support the Government’s mortgage guarantee scheme. You can apply for a first time buyer mortgage of up to 95% of the property's value.",
 "Use our mortgage calculator to help work out how much your mortgage repayments could be once you’ve paid a deposit. How much can I borrow as a first time buyer? Mortgage lenders may let you borrow up to 4.5 times your annual income. This can strike a balance between giving you the flexibility to choose the home that fits your needs and being able to keep up with repayments. It's important to budget for all your outgoings and living costs 

In [25]:
llama_cpp_generator.generate_chat_input(query, documents)

[{'role': 'system', 'content': 'you are a helpful mortgage advisor'},
 {'role': 'user',
  'content': "\n            DOCUMENTS:\n            \n             - You might come across the term first time buyer mortgage, or FTB mortgage. This is how we talk about mortgages for people who are buying a home for the first time. There are also some government mortgage schemes that are specifically for first time buyers. For example, the mortgage guarantee scheme may allow you to apply for a mortgage with a 5% deposit. Visit our first time buyer mortgage hub \n\n            \n             - We're proud to support the Government’s mortgage guarantee scheme. You can apply for a first time buyer mortgage of up to 95% of the property's value. \n\n            \n             - Use our mortgage calculator to help work out how much your mortgage repayments could be once you’ve paid a deposit. How much can I borrow as a first time buyer? Mortgage lenders may let you borrow up to 4.5 times your annual inco

In [26]:
response = llama_cpp_generator.run(query, documents)

In [ ]:
## Need to do more data collection and start preparing the presentation

In [27]:
print(response)

Yes, you can apply for a First Time Buyer mortgage. According to the documents, you can apply for a first time buyer mortgage of up to 95% of the property's value under the Government’s mortgage guarantee scheme. The eligibility criteria include:

- You must be a first-time buyer or home mover
- You need to have a deposit of at least 5% but less than 10%
- You can borrow less than £570,000 for a property that is not a new build flat, Shared Ownership, Shared Equity, Right to Buy, or buy to let
- You must apply for a qualifying mortgage by June 30, 2025
- Your mortgage application must be completed by December 31, 2025
- You can apply for a mortgage with a loan-to-value (LTV) of more than 90% and less than or equal to 95%

To apply, you should follow the usual application process outlined in the documents.
